# Criando um Web App de Análise de Sentimento
## Usando PyTorch e SageMaker

_Esse tutorial foi feito com base no projeto final do Nanodegree em Deep Learning da Udacity_

---

Agora que vocês já conhecem um pouco de AWS e de Sagemaker, iremos construir um projeto completo (end to end) com o intuito de desenvolver um web app onde o usuário insira um texto de avaliação de filme e saiba se essa avaliação é positiva ou negativa.

## Linhas Gerais


Esboço geral para projetos SageMaker usando uma instância de notebook.

1. Baixe ou recupere os dados.
2. Processe / prepare os dados.
3. Faça upload dos dados processados para S3.
4. Treine um modelo escolhido.
5. Teste o modelo treinado (normalmente usando um trabalho de transformação em lote).
6. Implante o modelo treinado.
7. Use o modelo implantado.

Para este projeto, seguiremos todas as etapas gerais

In [1]:
# Make sure that we use SageMaker 1.x
!pip install sagemaker==1.72.0

     |████████████████████████████████| 297 kB 4.3 MB/s eta 0:00:01
  Created wheel for sagemaker: filename=sagemaker-1.72.0-py2.py3-none-any.whl size=388346 sha256=3df83142c25b5950de80faee5f73b95a86ecb9d029894e13cc37b24e126e48d9
  Stored in directory: /home/ec2-user/.cache/pip/wheels/c3/58/70/85faf4437568bfaa4c419937569ba1fe54d44c5db42406bbd7
Successfully built sagemaker
  Attempting uninstall: smdebug-rulesconfig
    Found existing installation: smdebug-rulesconfig 1.0.1
    Uninstalling smdebug-rulesconfig-1.0.1:
      Successfully uninstalled smdebug-rulesconfig-1.0.1
  Attempting uninstall: sagemaker
    Found existing installation: sagemaker 2.68.0
    Uninstalling sagemaker-2.68.0:
      Successfully uninstalled sagemaker-2.68.0
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/pytorch_p36/bin/python -m pip install --upgrade pip' command.


## Passo 1: Baixando os dados

Para o nosso projeto, usaremos uma base bastante conhecida chamada [IMDb dataset](http://ai.stanford.edu/~amaas/data/sentiment/).

> Maas, Andrew L., et al. [Learning Word Vectors for Sentiment Analysis](http://ai.stanford.edu/~amaas/data/sentiment/). In _Proceedings of the 49th Annual Meeting of the Association for Computational Linguistics: Human Language Technologies_. Association for Computational Linguistics, 2011.

In [2]:
%mkdir ../data
!wget -O ../data/aclImdb_v1.tar.gz http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -zxf ../data/aclImdb_v1.tar.gz -C ../data

mkdir: cannot create directory ‘../data’: File exists
--2021-12-01 04:31:48--  http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
Resolving ai.stanford.edu (ai.stanford.edu)... 171.64.68.10
Connecting to ai.stanford.edu (ai.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84125825 (80M) [application/x-gzip]
Saving to: ‘../data/aclImdb_v1.tar.gz’

../data/aclImdb_v1. 100%[===================>]  80.23M  34.4MB/s    in 2.3s    

2021-12-01 04:31:50 (34.4 MB/s) - ‘../data/aclImdb_v1.tar.gz’ saved [84125825/84125825]



## Passo 2: Preparando e processando os dados

Processaremos os nossos dados a fim de deixá-los em um formato mais fácil para realizar o treinamento do modelo. Primeiramente, iremos unir as avaliação positivas e negativas em uma mesma estrutura de dados, uma vez que os dados vem em arquivos separados. Após isso iremos separá-los em treino e teste, garantindo que eles estejam misturados.

In [3]:
import os
import glob

def read_imdb_data(data_dir='../data/aclImdb'):
    data = {}
    labels = {}
    
    for data_type in ['train', 'test']:
        data[data_type] = {}
        labels[data_type] = {}
        
        for sentiment in ['pos', 'neg']:
            data[data_type][sentiment] = []
            labels[data_type][sentiment] = []
            
            path = os.path.join(data_dir, data_type, sentiment, '*.txt')
            files = glob.glob(path)
            
            for f in files:
                with open(f) as review:
                    data[data_type][sentiment].append(review.read())
                    # Here we represent a positive review by '1' and a negative review by '0'
                    labels[data_type][sentiment].append(1 if sentiment == 'pos' else 0)
                    
            assert len(data[data_type][sentiment]) == len(labels[data_type][sentiment]), \
                    "{}/{} data size does not match labels size".format(data_type, sentiment)
                
    return data, labels

In [4]:
data, labels = read_imdb_data()
print("IMDB reviews: train = {} pos / {} neg, test = {} pos / {} neg".format(
            len(data['train']['pos']), len(data['train']['neg']),
            len(data['test']['pos']), len(data['test']['neg'])))

IMDB reviews: train = 12500 pos / 12500 neg, test = 12500 pos / 12500 neg


Agora que lemos os dados brutos de treinamento e teste do conjunto de dados baixado, combinaremos as avaliações positivas e negativas e embaralharemos os registros resultantes.

In [5]:
from sklearn.utils import shuffle

def prepare_imdb_data(data, labels):
    """Prepare training and test sets from IMDb movie reviews."""
    
    #Combine positive and negative reviews and labels
    data_train = data['train']['pos'] + data['train']['neg']
    data_test = data['test']['pos'] + data['test']['neg']
    labels_train = labels['train']['pos'] + labels['train']['neg']
    labels_test = labels['test']['pos'] + labels['test']['neg']
    
    #Shuffle reviews and corresponding labels within training and test sets
    data_train, labels_train = shuffle(data_train, labels_train)
    data_test, labels_test = shuffle(data_test, labels_test)
    
    # Return a unified training data, test data, training labels, test labets
    return data_train, data_test, labels_train, labels_test

In [6]:
train_X, test_X, train_y, test_y = prepare_imdb_data(data, labels)
print("IMDb reviews (combined): train = {}, test = {}".format(len(train_X), len(test_X)))

IMDb reviews (combined): train = 25000, test = 25000


Agora, vamos dar uma olhadinha nos nossos dados!

In [7]:
print(train_X[100])
print(train_y[100])

This, despite not being the original - it began life as a play in Central Europe - has weathered the several incarnations that followed (MGM's own remake with period songs In The Good Old Summertime, the Broadway show She Loves Me, even the excellent theatre revival in Paris a couple of years ago) and remains the definitive version and the one they all have to beat. Several previous commenters have identified the contributing factors that make it so successful and memorable not least being the prevailing fashion in 30s and 40s Hollywood for lavishing attention and detail on ensemble playing rather than just two leads as so often happens today - try, for example, removing Ugarte, Ferrari, Renault etc from Casablanca and yes you'd still have Rick and Ilsa and Viktor Lazslo but they'd just be frosting without the rich cake mixture below. Jimmy Stewart and Maggie Sullavan WERE both ideal and irreplaceable leads but how much brighter they shine when their performances are reflected in those

Como primeiro passo de pré processamento, iremos limpar as tags HTML que podem aparecer nas avaliações e, após isso, iremos _tokenizar_ nossos dados para que palavras como *entertained* e *entertaining* sejam consideradas iguais no nosso modelo.

In [8]:
import nltk
from nltk.stem.porter import *

import re
from bs4 import BeautifulSoup

def review_to_words(review):
    stopwords = ["i", "me", "my", "myself", "we", "our", "ours", "ourselves", "you", "your", "yours", "yourself", "yourselves", "he", "him", "his", "himself", "she", "her", "hers", "herself", "it", "its", "itself", "they", "them", "their", "theirs", "themselves", "what", "which", "who", "whom", "this", "that", "these", "those", "am", "is", "are", "was", "were", "be", "been", "being", "have", "has", "had", "having", "do", "does", "did", "doing", "a", "an", "the", "and", "but", "if", "or", "because", "as", "until", "while", "of", "at", "by", "for", "with", "about", "against", "between", "into", "through", "during", "before", "after", "above", "below", "to", "from", "up", "down", "in", "out", "on", "off", "over", "under", "again", "further", "then", "once", "here", "there", "when", "where", "why", "how", "all", "any", "both", "each", "few", "more", "most", "other", "some", "such", "no", "nor", "not", "only", "own", "same", "so", "than", "too", "very", "s", "t", "can", "will", "just", "don", "should", "now"]
    stemmer = PorterStemmer()
    
    text = BeautifulSoup(review, "html.parser").get_text() # Remove HTML tags
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower()) # Convert to lower case
    words = text.split() # Split string into words
    words = [w for w in words if w not in stopwords] # Remove stopwords
    words = [PorterStemmer().stem(w) for w in words] # stem
    
    return words

A função `review_to_words` utiliza a biblioteca `BeautifulSoup` para remover as tags HTML e usa a biblioteca `nltk` para tokenizar as avaliações e remover as stopwords.  
Abaixo, podemos ver o output dessa função aplicado a uma avaliação.

In [9]:
review_to_words(train_X[100])

['despit',
 'origin',
 'began',
 'life',
 'play',
 'central',
 'europ',
 'weather',
 'sever',
 'incarn',
 'follow',
 'mgm',
 'remak',
 'period',
 'song',
 'good',
 'old',
 'summertim',
 'broadway',
 'show',
 'love',
 'even',
 'excel',
 'theatr',
 'reviv',
 'pari',
 'coupl',
 'year',
 'ago',
 'remain',
 'definit',
 'version',
 'one',
 'beat',
 'sever',
 'previou',
 'comment',
 'identifi',
 'contribut',
 'factor',
 'make',
 'success',
 'memor',
 'least',
 'prevail',
 'fashion',
 '30',
 '40',
 'hollywood',
 'lavish',
 'attent',
 'detail',
 'ensembl',
 'play',
 'rather',
 'two',
 'lead',
 'often',
 'happen',
 'today',
 'tri',
 'exampl',
 'remov',
 'ugart',
 'ferrari',
 'renault',
 'etc',
 'casablanca',
 'ye',
 'd',
 'still',
 'rick',
 'ilsa',
 'viktor',
 'lazslo',
 'd',
 'frost',
 'without',
 'rich',
 'cake',
 'mixtur',
 'jimmi',
 'stewart',
 'maggi',
 'sullavan',
 'ideal',
 'irreplac',
 'lead',
 'much',
 'brighter',
 'shine',
 'perform',
 'reflect',
 'frank',
 'morgan',
 'felix',
 'bressa

Já a função abaixo, `preprocess_data`, aplica a função `review_to_words` para cada uma das avaliações dos datasets de treino e teste. Além disso, ela também faz o cache dos dados, para que, caso algo aconteça, você possa voltar o pré processamento de onde parou.

In [10]:
import pickle

cache_dir = os.path.join("../cache", "sentiment_analysis")  # where to store cache files
os.makedirs(cache_dir, exist_ok=True)  # ensure cache directory exists

def preprocess_data(data_train, data_test, labels_train, labels_test,
                    cache_dir=cache_dir, cache_file="preprocessed_data.pkl"):
    """Convert each review to words; read from cache if available."""

    # If cache_file is not None, try to read from it first
    cache_data = None
    if cache_file is not None:
        try:
            with open(os.path.join(cache_dir, cache_file), "rb") as f:
                cache_data = pickle.load(f)
            print("Read preprocessed data from cache file:", cache_file)
        except:
            pass  # unable to read from cache, but that's okay
    
    # If cache is missing, then do the heavy lifting
    if cache_data is None:
        # Preprocess training and test data to obtain words for each review
        #words_train = list(map(review_to_words, data_train))
        #words_test = list(map(review_to_words, data_test))
        words_train = [review_to_words(review) for review in data_train]
        words_test = [review_to_words(review) for review in data_test]
        
        # Write to cache file for future runs
        if cache_file is not None:
            cache_data = dict(words_train=words_train, words_test=words_test,
                              labels_train=labels_train, labels_test=labels_test)
            with open(os.path.join(cache_dir, cache_file), "wb") as f:
                pickle.dump(cache_data, f)
            print("Wrote preprocessed data to cache file:", cache_file)
    else:
        # Unpack data loaded from cache file
        words_train, words_test, labels_train, labels_test = (cache_data['words_train'],
                cache_data['words_test'], cache_data['labels_train'], cache_data['labels_test'])
    
    return words_train, words_test, labels_train, labels_test

In [11]:
# Preprocess data
train_X, test_X, train_y, test_y = preprocess_data(train_X, test_X, train_y, test_y)

Read preprocessed data from cache file: preprocessed_data.pkl


### Transformando os dados

Agora, nós iremos construir uma representação dos nossos dados muito similar a representação conhecida como bag-of-words. Para a rede neural recorrente que iremos usar, nós iremos relizar a transformação dos dados da seguinte forma:

1. Transformar cada palavra em um número inteiro;
2. Definir um tamanho para nosso vocabulário, ou seja, iremos remover palavras que aparecem pouco (para essas palavras atribuiremos o mesmo número inteiro (1));
3. Como estamos usando uma RNN, defineros um tamanho para nossas sequência, ou seja, truncaremos aquelas que forem maiores e iremos inserir um caractér (0) para quando a avaliação for menor do que o tamanho definido.

Começaremos construindo uma função que nos retorna um dicionário de tamanho especifíco e com as palavras que mais aparecem. Não podemos esquecer de reservar o índice 0 e 1 para os caractéres vazio e pouco frequente!

In [12]:
import numpy as np

def build_dict(data, vocab_size = 5000):
    """Construct and return a dictionary mapping each of the most frequently appearing words to a unique integer."""
    
    word_count = {} # A dict storing the words that appear in the reviews along with how often they occur
    for review in data:
        for word in review:
            if word_count.get(word) is None:
                word_count[word] = 1
            else:
                word_count[word] += 1
    
    sorted_words = [k for k,v in sorted(word_count.items(), key=lambda item: item[1], reverse=True)]
    
    word_dict = {} # This is what we are building, a dictionary that translates words into integers
    for idx, word in enumerate(sorted_words[:vocab_size - 2]): # The -2 is so that we save room for the 'no word'
        word_dict[word] = idx + 2                              # 'infrequent' labels
        
    return word_dict

In [13]:
word_dict = build_dict(train_X)

### Salvando nosso dicionário

Mais para frente, quando tivermos nosso modelo, teremos que usar nosso dicionário para realizar as predições. Sendo assim, precisamos salvá-lo para usar no futuro!

In [14]:
data_dir = '../data/pytorch' # The folder we will use for storing data
if not os.path.exists(data_dir): # Make sure that the folder exists
    os.makedirs(data_dir)

In [15]:
with open(os.path.join(data_dir, 'word_dict.pkl'), "wb") as f:
    pickle.dump(word_dict, f)

### Transformando as avaliações

Agora, é hora de convertermos nossas avaliações de treino e teste para a sequência de número inteiros de tamanho fixo que entrará na nossa rede neural!

In [16]:
def convert_and_pad(word_dict, sentence, pad=500):
    NOWORD = 0 # We will use 0 to represent the 'no word' category
    INFREQ = 1 # and we use 1 to represent the infrequent words, i.e., words not appearing in word_dict
    
    working_sentence = [NOWORD] * pad
    
    for word_index, word in enumerate(sentence[:pad]):
        if word in word_dict:
            working_sentence[word_index] = word_dict[word]
        else:
            working_sentence[word_index] = INFREQ
            
    return working_sentence, min(len(sentence), pad)

def convert_and_pad_data(word_dict, data, pad=500):
    result = []
    lengths = []
    
    for sentence in data:
        converted, leng = convert_and_pad(word_dict, sentence, pad)
        result.append(converted)
        lengths.append(leng)
        
    return np.array(result), np.array(lengths)

In [17]:
train_X, train_X_len = convert_and_pad_data(word_dict, train_X)
test_X, test_X_len = convert_and_pad_data(word_dict, test_X)

Apenas para validar se nossa função está fazendo tudo corretamente, vamos verificar um dado da nossa base de treino e ver seu tamanho.

In [18]:
# Use this cell to examine one of the processed reviews to make sure everything is working as intended.
print(train_X[100])
print(len(train_X[100]))

[4340    2   25   24   69 2295  128   31   65  220   44  176  125  206
   17  118  504  123   11   16   49   11  267  843  694  843   18    7
 1653    2  862    1  205  176   20 2494 2794 1099 1183   38   95 3932
 2433  761 2494 2794 1670 3932  279  308    1  161    1    1  135  851
 2028   17  233  208   74   54  213 2295  128    5  220  145  296    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0 

## Passo 3: Subindo nossos dados de treino para o S3

Nós iremos precisar subir nossos dados de treino para o S3 para que possamos acessá-lo durante o treinamento.

### Salvando os dados de treino localmente

Antes de subir para o S3, iremos salvar nossos dados localmente. É muito importante saber a estrutura dos dados que vamos salvar, para que possamos utilizar de forma correta. No nosso caso, as linhas do nosso dataset irão ter a forma (colunas): `label`, `length`, `review[500]`, onde `review[500]`é a sequência com 500 números inteiros que geramos acima.

In [19]:
import pandas as pd
    
pd.concat([pd.DataFrame(train_y), pd.DataFrame(train_X_len), pd.DataFrame(train_X)], axis=1) \
        .to_csv(os.path.join(data_dir, 'train.csv'), header=False, index=False)

### Subindo para o S3

In [20]:
import sagemaker

sagemaker_session = sagemaker.Session()

bucket = sagemaker_session.default_bucket()
prefix = 'sagemaker/sentiment_rnn'

role = sagemaker.get_execution_role()

In [21]:
input_data = sagemaker_session.upload_data(path=data_dir, bucket=bucket, key_prefix=prefix)

**OBS:** A célula acima sobe todos os arquivos contidos no nosso diretório para o S3. Isso inclui o `word_dict.pkl`, que usaremos na hora de realizar uma nova predição e garantir que o pré processamento dos novos dados seja o mesmo dos dados de treino. 

## Passo 4: Construindo e treinando nosso modelo em Pytorch


Em particular, um modelo compreende três objetos

 - Artefatos de modelo,
 - Código de Treinamento e
 - Código de inferência,
 
cada um dos quais interage um com o outro. Implementaremos nossa própria rede neural no PyTorch junto com um script de treinamento. Para os fins deste projeto, fornecemos o objeto de modelo necessário no arquivo `model.py`, dentro da pasta `train`. Você pode ver a implementação fornecida executando a célula abaixo.

In [22]:
!pygmentize train/model.py

import torch.nn as nn

class LSTMClassifier(nn.Module):
    """
    This is the simple RNN model we will be using to perform Sentiment Analysis.
    """

    def __init__(self, embedding_dim, hidden_dim, vocab_size):
        """
        Initialize the model by settingg up the various layers.
        """
        super(LSTMClassifier, self).__init__()

        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)
        self.dense = nn.Linear(in_features=hidden_dim, out_features=1)
        self.sig = nn.Sigmoid()
        
        self.word_dict = None

    def forward(self, x):
        """
        Perform a forward pass of our model on some input.
        """
        x = x.t()
        lengths = x[0,:]
        reviews = x[1:,:]
        embeds = self.embedding(reviews)
        lstm_out, _ = self.lstm(embeds)
        out = self.dense(lstm_out)
        out = out[lengths - 1, range(len(lengths))]
        return self.sig

A conclusão importante da implementação fornecida é que existem três parâmetros que podemos desejar ajustar para melhorar o desempenho de nosso modelo. Estas são a dimensão de embedding, a dimensão oculta e o tamanho do vocabulário. Provavelmente, desejaremos tornar esses parâmetros configuráveis no script de treinamento para que, se desejarmos modificá-los, não precisemos modificar o próprio script. Para começar, escreveremos parte do código de treinamento no notebook para que possamos diagnosticar mais facilmente quaisquer problemas que surjam.

Primeiro, carregaremos uma pequena parte do conjunto de dados de treinamento para usar como amostra. Seria muito demorado tentar treinar o modelo completamente no notebook, pois não temos acesso a uma gpu e a instância de computação que estamos usando não é particularmente poderosa. No entanto, podemos trabalhar com alguns dados para ter uma ideia de como nosso script de treinamento está se comportando.

In [23]:
import torch
import torch.utils.data

# Read in only the first 250 rows
train_sample = pd.read_csv(os.path.join(data_dir, 'train.csv'), header=None, names=None, nrows=250)

# Turn the input pandas dataframe into tensors
train_sample_y = torch.from_numpy(train_sample[[0]].values).float().squeeze()
train_sample_X = torch.from_numpy(train_sample.drop([0], axis=1).values).long()

# Build the dataset
train_sample_ds = torch.utils.data.TensorDataset(train_sample_X, train_sample_y)
# Build the dataloader
train_sample_dl = torch.utils.data.DataLoader(train_sample_ds, batch_size=50)

###  Escrevendo o método de treinamento

Em seguida, precisamos escrever o código de treinamento. Isso deve ser muito semelhante aos métodos de treinamento que escrevemos antes para treinar modelos em PyTorch. Vamos deixar todos os aspectos difíceis, como salvar / carregar o modelo e carregar os parâmetros, para um pouco mais tarde.

In [24]:
def train(model, train_loader, epochs, optimizer, loss_fn, device):
    for epoch in range(1, epochs + 1):
        model.train()
        total_loss = 0
        for batch in train_loader:         
            batch_X, batch_y = batch
            
            batch_X = batch_X.to(device)
            batch_y = batch_y.to(device)
            
            optimizer.zero_grad()
            output = model.forward(batch_X)
            loss = loss_fn(output, batch_y)
            loss.backward()
            optimizer.step()
            
            total_loss += loss.data.item()
        print("Epoch: {}, BCELoss: {}".format(epoch, total_loss / len(train_loader)))

Supondo que temos o método de treinamento acima, testaremos se ele está funcionando escrevendo um pouco de código no notebook que executa nosso método de treinamento no pequeno conjunto de treinamento de amostra que carregamos anteriormente. A razão para fazer isso no notebook é para que tenhamos a oportunidade de corrigir quaisquer erros que surjam no início, quando são mais fáceis de diagnosticar.

In [25]:
import torch.optim as optim
from train.model import LSTMClassifier

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = LSTMClassifier(32, 100, 5000).to(device)
optimizer = optim.Adam(model.parameters())
loss_fn = torch.nn.BCELoss()

train(model, train_sample_dl, 5, optimizer, loss_fn, device)

Epoch: 1, BCELoss: 0.6916054129600525
Epoch: 2, BCELoss: 0.6835224390029907
Epoch: 3, BCELoss: 0.6764762163162231
Epoch: 4, BCELoss: 0.6685465693473815
Epoch: 5, BCELoss: 0.658632218837738


Para construir um modelo PyTorch usando o SageMaker, devemos fornecer ao SageMaker um script de treinamento. Podemos opcionalmente incluir um diretório que será copiado para o contêiner e a partir do qual nosso código de treinamento será executado. Quando o contêiner de treinamento é executado, ele verifica o diretório carregado (se houver) para um arquivo `requirements.txt` e instala todas as bibliotecas Python necessárias, após o qual o script de treinamento será executado.

### Treinando o modelo

Quando um modelo PyTorch é construído no SageMaker, um ponto de entrada deve ser especificado. Este é o arquivo Python que será executado quando o modelo for treinado. Dentro do diretório `train` está um arquivo chamado` train.py` que é fornecido e que contém o código necessário para treinar nosso modelo. 

A maneira como o SageMaker passa hiperparâmetros para o script de treinamento é por meio de argumentos. Esses argumentos podem ser analisados e usados no script de treinamento. Para ver como isso é feito, dê uma olhada no arquivo `train / train.py` fornecido.

In [26]:
from sagemaker.pytorch import PyTorch

estimator = PyTorch(entry_point="train.py",
                    source_dir="train",
                    role=role,
                    framework_version='0.4.0',
                    train_instance_count=1,
                    train_instance_type='ml.c5.xlarge',
                    hyperparameters={
                        'epochs': 3,
                        'hidden_dim': 200,
                    })

In [27]:
estimator.fit({'training': input_data})

'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.
's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.
'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.


2021-12-01 04:32:13 Starting - Starting the training job...
2021-12-01 04:32:14 Starting - Launching requested ML instances...
2021-12-01 04:33:03 Starting - Preparing the instances for training.........
2021-12-01 04:34:41 Downloading - Downloading input data
2021-12-01 04:34:41 Training - Training image download completed. Training in progress..bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2021-12-01 04:34:41,999 sagemaker-containers INFO     Imported framework sagemaker_pytorch_container.training
2021-12-01 04:34:42,001 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2021-12-01 04:34:42,016 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2021-12-01 04:34:48,234 sagemaker_pytorch_container.training INFO     Invoking user training script.
2021-12-01 04:34:48,413 sagemaker-containers INFO     Module train does not provide a setup.py. 
Generating setup.p

## Passo 5: Implantando o modelo em um aplicativo da web

Agora que sabemos que nosso modelo está funcionando, é hora de criar algum código de inferência personalizado para que possamos enviar ao modelo uma revisão que não foi processada e determinar o sentimento da revisão.

Como vimos acima, por padrão, o estimador que criamos, quando implantado, usará o script de entrada e o diretório que fornecemos ao criar o modelo. No entanto, como agora desejamos aceitar uma string como entrada e nosso modelo espera uma revisão processada, precisamos escrever algum código de inferência personalizado.

Vamos armazenar o código que escrevemos no diretório `serve`. Fornecido neste diretório está o arquivo `model.py` que usamos para construir nosso modelo, um arquivo` utils.py` que contém as funções de pré-processamento `review_to_words` e` convert_and_pad` que usamos durante o processamento inicial de dados, e `Predict.py`, o arquivo que conterá nosso código de inferência personalizado. Observe também que `requirements.txt` está presente, o que dirá ao SageMaker quais bibliotecas Python são exigidas por nosso código de inferência personalizado.

Ao implantar um modelo PyTorch no SageMaker, espera-se que você forneça quatro funções que o contêiner de inferência SageMaker usará.
 - `model_fn`: Esta função é a mesma função que usamos no script de treinamento e diz ao SageMaker como carregar nosso modelo.
 - `input_fn`: esta função recebe a entrada serializada bruta que foi enviada para o endpoint do modelo e seu trabalho é desserializar e disponibilizar a entrada para o código de inferência.
 - `output_fn`: esta função pega a saída do código de inferência e seu trabalho é serializar esta saída e retorná-la ao chamador do endpoint do modelo.
 - `Predict_fn`: O coração do script de inferência, é aqui que a previsão real é feita e é a função que você precisa completar.

Para o site simples que estamos construindo durante este projeto, os métodos `input_fn` e` output_fn` são relativamente diretos. Só exigimos ser capazes de aceitar uma string como entrada e esperamos retornar um único valor como saída. Você pode imaginar, entretanto, que em um aplicativo mais complexo, a entrada ou saída podem ser dados de imagem ou alguns outros dados binários que exigiriam algum esforço para serializar.


### Escrevendo código de inferência

Começaremos dando uma olhada no código que foi fornecido.

In [29]:
!pygmentize serve/predict.py

import argparse
import json
import os
import pickle
import sys
import sagemaker_containers
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data

from model import LSTMClassifier

from utils import review_to_words, convert_and_pad

def model_fn(model_dir):
    """Load the PyTorch model from the `model_dir` directory."""
    print("Loading model.")

    # First, load the parameters used to create the model.
    model_info = {}
    model_info_path = os.path.join(model_dir, 'model_info.pth')
    with open(model_info_path, 'rb') as f:
        model_info = torch.load(f)

    print("model_info: {}".format(model_info))

    # Determine the device and construct the model.
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = LSTMClassifier(model_info['embedding_dim'], model_info['hidden_dim'], model_info['vocab_size'])

    # Load the store model parameters.
    model_path = os.path.join(mod

Como mencionado anteriormente, o método `model_fn` é o mesmo fornecido no código de treinamento e os métodos` input_fn` e `output_fn` são muito simples e sua tarefa será completar o método` predict_fn`. Certifique-se de salvar o arquivo completo como `predict.py` no diretório` serve`.

### Implantando o modelo

Agora que o código de inferência personalizado foi escrito, criaremos e implantaremos nosso modelo. Para começar, precisamos construir um novo objeto PyTorchModel que aponta para os artefatos do modelo criados durante o treinamento e também aponta para o código de inferência que desejamos usar. Em seguida, podemos chamar o método de implantação para iniciar o contêiner de implantação.

** NOTA **: O comportamento padrão para um modelo PyTorch implantado é assumir que qualquer entrada passada ao preditor é um array `numpy`. Em nosso caso, queremos enviar uma string, então precisamos construir um wrapper simples em torno da classe `RealTimePredictor` para acomodar strings simples. Em uma situação mais complicada, você pode desejar fornecer um objeto de serialização, por exemplo, se desejar enviar dados de imagem.

In [30]:
from sagemaker.predictor import RealTimePredictor
from sagemaker.pytorch import PyTorchModel

class StringPredictor(RealTimePredictor):
    def __init__(self, endpoint_name, sagemaker_session):
        super(StringPredictor, self).__init__(endpoint_name, sagemaker_session, content_type='text/plain')

model = PyTorchModel(model_data=estimator.model_data,
                     role = role,
                     framework_version='0.4.0',
                     entry_point='predict.py',
                     source_dir='serve',
                     predictor_cls=StringPredictor)
predictor = model.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

Parameter image will be renamed to image_uri in SageMaker Python SDK v2.
'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.


------!

### Testando o modelo

Agora que implantamos nosso modelo com o código de inferência personalizado, devemos testar para ver se tudo está funcionando. Aqui, testamos nosso modelo carregando os primeiros `250` comentários positivos e negativos e os enviamos para o endpoint, em seguida, coletamos os resultados. O motivo para enviar apenas alguns dos dados é que o tempo que leva para nosso modelo processar a entrada e, em seguida, realizar a inferência é muito longo e, portanto, testar todo o conjunto de dados seria proibitivo.

In [31]:
import glob

def test_reviews(data_dir='../data/aclImdb', stop=250):
    
    results = []
    ground = []
    
    # We make sure to test both positive and negative reviews    
    for sentiment in ['pos', 'neg']:
        
        path = os.path.join(data_dir, 'test', sentiment, '*.txt')
        files = glob.glob(path)
        
        files_read = 0
        
        print('Starting ', sentiment, ' files')
        
        # Iterate through the files and send them to the predictor
        for f in files:
            with open(f) as review:
                # First, we store the ground truth (was the review positive or negative)
                if sentiment == 'pos':
                    ground.append(1)
                else:
                    ground.append(0)
                # Read in the review and convert to 'utf-8' for transmission via HTTP
                review_input = review.read().encode('utf-8')
                # Send the review to the predictor and store the results
                results.append(float(predictor.predict(review_input)))
                
            # Sending reviews to our endpoint one at a time takes a while so we
            # only send a small number of reviews
            files_read += 1
            if files_read == stop:
                break
            
    return ground, results

In [32]:
ground, results = test_reviews()

Starting  pos  files
Starting  neg  files


In [33]:
from sklearn.metrics import accuracy_score
accuracy_score(ground, results)

0.772

Agora que sabemos que nosso endpoint está funcionando conforme o esperado, podemos configurar a página da web que irá interagir com ele.

## Passo 6: Criando nossa função Lambda

Construimos nosso endpoint e deployamos nosso modelo, mas... e se quisessemos que nosso modelo fosse acessável por meio de um webapp? Para isso, precisamos construir alguns outros componentes, que podemos ver na arquitetura abaixo:

<img src="../deploy-aws.png">

Indo da esquerda para direita temos: 

- A EC2 será responsável por subir nossa aplicação Flask, servir nossa página e enviar a requisição para o API Gateway;
- O API Gateway receberá a requisição e encaminhará para a função Lambda;
- Já a função Lambda funcionará como redirecionando o texto da nova avaliação para o Sagemaker endpoint, uma vez que não há como realizar a integração API Gateway -> Sagemaker Endpoint de forma nativa e também não podemos chamar a URL do Sagemaker Endpoint sem estar dentro da AWS.

### Configurando nossa função Lambda

A primeira coisa que faremos será a criação da nossa função Lambda. Nossa função receberá como input os dados vindos do nosso API Gateway, irá realizar a chamada do endpoint e retornará a resposta para o API Gateway.

#### Parte A: Criando IAM Role para a Lambda

Já que queremos que nossa função Lambda chame o Sagemaker Endpoint, precisamos garantir que ele tenha permissão para isso. Dessa forma, precisamos adicionar essa permissão dentro da Role que usaremos em nossa função Lambda.

Usando o Console da AWS, procuraremos por **IAM** na barra de busca e clicaremos em **Roles** no menu esquerdo. Feito isso, clique em **Create Role**. Garante que em **AWS service** o time de _trusted entity_ selecionado seja **Lambda** e, em seguida, clique em **Next: Permissions**.

Na barra de busca procure por `sagemaker` e clique no checkbox referente a **AmazonSageMakerFullAccess** policy. Clique em **Next: Review**.

Por último, dê um nome para usa role e garanta que você irá se lembrar na hora de criar sua função Lambda!

Usaremos o nome `LambdaSageMakerRole`.

#### Parte B: Criando a Lambda

Agora, é hora de criarmos nossa função Lambda!

Para isso, no console AWS, procure por Lambda e clique em **Create a function**. Na página seguinte, clique em **Author from scratch**, selecione o runtime como sendo Python, dê um nome para sua função, como por exemplo: `sentiment_analysis_func`. Não esqueça de selecionar a role que criamos!

Após isso, clique em **Create Function**.

Na próxima página você verá algumas informações sobre sua função Lambda que você acabou de criar. Se você descer um pouco a página, verá um editor de texto onde você pode escrever código que será executado quando sua função for chamada. No nosso projeto, usaremos o código abaixo (basta copiar e colar) e não esqueça de mudar o `EndpointName` para o nome do endpoint que pegaremos na célula abaixo.

```python
# We need to use the low-level library to interact with SageMaker since the SageMaker API
# is not available natively through Lambda.
import boto3

def lambda_handler(event, context):

    # The SageMaker runtime is what allows us to invoke the endpoint that we've created.
    runtime = boto3.Session().client('sagemaker-runtime')

    # Now we use the SageMaker runtime to invoke our endpoint, sending the review we were given
    response = runtime.invoke_endpoint(EndpointName = '**ENDPOINT NAME HERE**',    # The name of the endpoint we created
                                       ContentType = 'text/plain',                 # The data format that is expected
                                       Body = event['body'])                       # The actual review

    # The response is an HTTP response whose body contains the result of our inference
    result = response['Body'].read().decode('utf-8')

    return {
        'statusCode' : 200,
        'headers' : { 'Content-Type' : 'text/plain', 'Access-Control-Allow-Origin' : '*' },
        'body' : result
    }
```

In [34]:
predictor.endpoint

'sagemaker-pytorch-2021-12-01-05-13-42-682'

Após alterar o nome do endpoint na sua função Lambda, clique em **Salvar**. Feito, sua função Lambda estará pronto para ser executada!

## Passo 7: Criando e configurando o API Gateway

Agora, é hora de criarmos nosso API Gateway!

Abra o console da AWS e busque por **API Gateway** e, na página inicial, procure por **REST API** (vão ter duas, selecione a que NÃO aparece private) e clique em **Build**. Na tela de configuração, na seção **Create new API** selecione **New API** e defina um nome para seu Gateway e clique em **Create API**.

Já dentro da sua API, precisamos criar nosso método POST, que receberá o texto a ser predito, e integrá-lo a função Lambda que criamos. Para isso, na aba **Resources** do menu lateral, clicaremos no botão **Actions** e selecionaremos a opção **Create Method**. Na lista dos nossos recursos aparecerá um dropdown vazio, clique nele e selecione **POST** e depois clique no &#x2611;.

Na tela de configuração do nosso método POST preencha da seguinte forma:

- **Integration type**:  Lambda Function;
- **Use Lambda Proxy integration**: marque essa checkbox;
- **Lambda Region**: Região que você criou seus outros recursos (no geral, já vem preenchido corretamente);
- **Lambda Function**: Nome da função Lambda que criamos no passo anterior.

Após preencher, clique em **Save**.

Para finalizar, precisamos deployar nosso API Gateway. Para isso, na aba **Resources** do menu lateral, clicamos no botão **Actions** e depois em **Deploy API**. Irá aparecer um pop-up no qual clicaremos em **Deployment stage**, selecionaremos a opção **[New Stage]** e depois daremos um nome a esse estágio (pode ser _prod_). Por fim, clicamos em **Deploy**.

Nessa janela que abrirá, você verá na parte superior a URL do seu API Gateway. Salve ela em algum lugar pois usaremos no passo a seguir, quando estivermos configurando nosso webapp!

## Step 8: Deployando nosso web app

Agora, como passo final, faremos o deploy do nosso web app, para que possamos acessá-lo pelo browser e saber o sentimento de novas avaliações de filmes!  
Para isso, usaremos uma instância de EC2 e subiremos uma aplicação Flask que será responsável por carregar a página e fazer as chamadas no API Gateway.

Comece procurando por **EC2** na barra de busca do console AWS. No menu lateral esquerdo clique em **Instances** e procure por um botão laranja, na parte superior direita, com o seguinte texto: **Launch instances**. Clique nele.

Após isso, você será redirecionado para uma tela onde terá que selecionar a AMI (imagem do sistema) que você irá utilizar em sua EC2. Selecione **Amazon Linux 2 AMI (HVM), SSD Volume Type** na versão **64-bit (x86)**. Ao clicar em **Select** você será redirecionado para a página onde poderá escolher o tipo da sua instância. Selecioner o tipo **t2.micro** e clique em **Next: Configure Instance Details**.

Na página seguinte, não precisa alterar nada, apenas clicar em **Next: Add Storage**. Em seguida, clique em **Next: Add tags** e depois em **Next: Configure Security Group**.

Nessa página configurar o `security group` da EC2 para garantir que seja permitido o ingresso pela porta 80 (Adicionar uma inbound rule permitindo HTTP para qualquer Ipv4). Para isso, clicamos em **Add rule**, mudamos o tipo para **HTTP** e o Source para **Anywhere**. Feito isso, clique em **Review and Launch** e depois em **Launch**.

Nessa hora, aparecerá um pop-up perdindo para você selecionar um par de chaves para fazer a conexão SSH. Não usaremos, pois iremos nos conectar à máquina via browser. Assim, selecione a opção **Proceed without a key pair**, marque o checkbox e clique em **Launch instances**.

Agora, temos que aguardar a máquina ficar com o status de pronta e, quando estiver tudo certo, clicamos em cima da nossa instância e depois em **Connect**.

Após isso, vamos acessar, via browser, o terminal da máquina que criamos e instalar o git (```yum install git```). Com o git instalado, iremos executar o comando ```https://github.com/vfcarida/NLP_TDC2021```. Não esqueça de, após clonar, entrar no diretório (```NLP-producao```).

Feito isso, iremos acessar a pasta `website\templates` e abrir o arquivo chamado `index.html` (```vim index.html```) para que possamos alterar a URL da nossa API. Procure por uma linha que contém **\*\*REPLACE WITH PUBLIC API URL\*\*** e insira a url da sua API onde será possível realizar a predição. Feito isso, salve o arquivo (```Esc -> :wq```) e volte para o diretório `website` (```cd ..```)`.

Já no repositório `website`, execute o comando ```pip3 install -r requirements.txt``` e, depois disso, o comando ```python3 -m flask run --host 0.0.0.0 --port 80```.

Pronto! Basta acessar o Ip público da sua máquina (fica na parte inferior da tela) e ver sua aplicação funcionando!

## Passo 9: Deletando a EC2 e o Sagemaker Endpoint

Não se esqueça de deletar os recursos assim que finalizar o uso, para que não gere cobranças desnecessárias.

Para apagar a EC2 procure por **EC2** na barra de busca do console AWS. No menu lateral esquerdo clique em **Instances** e depois selecione a instância que você criou para o nosso exemplo. Na parte superior, clique em **Instance State** e depois em **Terminate Instance**.

Já para o Sagemaker Endpoint, apenas rode o comando da célula abaixo.

OBS: Para deletar os outros recursos que criamos (Lambda e API Gateway) basta acessar tais produtos, selecionar o que você quer deletar e clicar em **Actions** e **Delete**.

In [ ]:
predictor.delete_endpoint()